Idea for solving workflow:
 1. define mesh using MeshHandler class
 2. define cell model using CellModel class
 3. define dynamics model using DynamicsModel class

## Importing packages

In [3]:
from dolfinx import fem, mesh, plot, io
from dolfinx.fem.petsc import LinearProblem
from configs import *
import pyvista
import utils
import numpy as np
import ufl
from mpi4py import MPI

## Create a mesh, elements and spaces


In [4]:
with io.XDMFFile(MPI.COMM_WORLD, "coarse_mesh.xdmf", 'r') as xdmf:
    domain = xdmf.read_mesh(name="Grid")

element = ufl.FiniteElement("P", domain.ufl_cell(), degree=2)
W = fem.FunctionSpace(domain, ufl.MixedElement(element, element))  # P1 FEM space
V1, sub1 = W.sub(0).collapse()
V2, sub2 = W.sub(1).collapse()

# Define test functions
psi, phi = ufl.TestFunctions(W)

# Define trialfunctions
V_m, U_e = ufl.TrialFunctions(W)

# Define functions
v_ = fem.Function(W)
w, w_n = fem.Function(V1), fem.Function(V1)
V_m_n = fem.Function(V1)

x = ufl.SpatialCoordinate(domain)
d = domain.topology.dim

RuntimeError: Unable to open HDF5 file. File coarse_mesh.h5 does not exist.

## Initial transmembrane potential

# Conductivities

In [ ]:
# Muscle fibres
fib_l = ufl.as_vector(
    [1, 0, 0]
    # [x[1] / (x[0] ** 2 + x[1] ** 2 + 1), -x[0] / (x[0] ** 2 + x[1] ** 2 + 1)]
)

fib_n = ufl.as_vector([0, 1, 0])

# Healthy conductivities
M_i_val = (
    SIGMA_IT * ufl.Identity(d)
    + (SIGMA_IL - SIGMA_IT) * ufl.outer(fib_l, fib_l)
    + (SIGMA_IN - SIGMA_IT) * ufl.outer(fib_n, fib_n)
)
M_e_val = (
    SIGMA_ET * ufl.Identity(d)
    + (SIGMA_EL - SIGMA_ET) * ufl.outer(fib_l, fib_l)
    + (SIGMA_EN - SIGMA_ET) * ufl.outer(fib_n, fib_n)
)

# Ishemic conductivities
tissue_location = (1.5, 1.5)
tissue_radius = 0.5

M_i = ufl.conditional(
    (x[0] - tissue_location[0]) ** 2 + (x[1] - tissue_location[1]) ** 2
    < tissue_radius**2,
    M_i_val / 10,
    M_i_val,
)

M_e = ufl.conditional(
    (x[0] - tissue_location[0]) ** 2 + (x[1] - tissue_location[1]) ** 2
    < tissue_radius**2,
    M_e_val / 10,
    M_e_val,
)

## Defining ufl expressions and a problem

In [ ]:
F = (V_m - V_m_n) / DT * phi * ufl.dx + ufl.inner(
    ufl.dot(M_i, ufl.grad(V_m / 2 + V_m_n / 2 + U_e)), ufl.grad(phi)
) * ufl.dx
F += (
    ufl.inner(ufl.dot(M_i + M_e, ufl.grad(U_e)), ufl.grad(psi)) * ufl.dx
    + ufl.inner(ufl.dot(M_i, ufl.grad(V_m / 2 + V_m_n / 2)), ufl.grad(psi)) * ufl.dx
)
problem = LinearProblem(a=ufl.lhs(F), L=ufl.rhs(F), u=v_)

### Setting up the plotting environment

In [ ]:
grid = pyvista.UnstructuredGrid(*plot.vtk_mesh(V1))
plotter = pyvista.Plotter(notebook=True, off_screen=False)
plotter.open_gif("V_m_time.gif", fps=int(NUM_STEPS / 10))
grid.point_data["V_m"] = V_m_n.x.array
plotter.add_mesh(
    grid,
    show_edges=False,
    lighting=True,
    smooth_shading=True,
    clim=[-100, 50],
)

## Full time-stepping:

In [2]:
# Defining a cell model
CellModel = utils.ReparametrizedFitzHughNagumo()

# List of signal values for each time step
signal = []
t = 0.0

# Iterate through time
while t < T:
    # Appending the transmembrane potential value at some point to a list
    #signal.append(utils.evaluate_function_at_point(V_m_n, domain, [1.5, 1.5, 0.0]))

    # 1st step of Strang splitting
    k1_V = CellModel.I_ion(V_m_n.x.array[:], w.x.array[:])
    k2_V = CellModel.I_ion(V_m_n.x.array[:] + DT / 2 * k1_V, w.x.array[:])
    V_m_n.x.array[:] = V_m_n.x.array[:] + DT / 4 * (k1_V + k2_V)

    k1_w = CellModel.f(V_m_n.x.array[:], w.x.array[:])
    k2_w = CellModel.f(V_m_n.x.array[:], w.x.array[:] + DT / 2 * k1_w)
    w.x.array[:] = w.x.array[:] + DT / 4 * (k1_w + k2_w)

    # 2nd step of Strang splitting
    problem.solve()
    v_.x.array[sub2] = v_.x.array[sub2] - np.mean(
        v_.x.array[sub2]
    )  # Normalize U_e to zero mean

    # Update solution for V_m
    V_m_n.x.array[:] = v_.x.array[sub1]

    # 3rd step of Strang splitting
    k1_V = CellModel.I_ion(V_m_n.x.array[:], w.x.array[:])
    k2_V = CellModel.I_ion(V_m_n.x.array[:] + DT * k1_V, w.x.array[:])
    V_m_n.x.array[:] = V_m_n.x.array[:] + DT / 2 * (k1_V + k2_V)

    k1_w = CellModel.f(V_m_n.x.array[:], w.x.array[:])
    k2_w = CellModel.f(V_m_n.x.array[:], w.x.array[:] + DT * k1_w)
    w.x.array[:] = w.x.array[:] + DT / 2 * (k1_w + k2_w)

    # Print time
    print("t = %.3f" % t)
    # Update plot
    plotter.clear()
    grid.point_data["V_m"] = V_m_n.x.array[:]
    plotter.add_mesh(
        grid,
        show_edges=False,
        lighting=True,
        smooth_shading=True,
        clim=[-100, 50],
    )
    plotter.add_title("t = %.3f" % t, font_size=24)
    plotter.view_vector([1, -1, -1])
    plotter.write_frame()
    t += DT

plotter.close()

NameError: name 'utils' is not defined

### Plotting time dependence of transmembrane potential at a particular point

In [ ]:
import matplotlib.pyplot as plt

time = np.linspace(0, T, len(signal))
plt.plot(time, signal)
plt.xlabel("time [ms]")
plt.ylabel("signal [mV]")
plt.title("Time dependence of transmembrane potential")

### Plotting a final plot of transmembrane potential

In [ ]:
utils.plot_function(V_m_n, V1, shadow=True)